#LSTM stock price prediction

In [3]:
#install required libraries 
!pip install yfinance
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# import libararies 
import math
import json
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM
import datetime as dt
from datetime import date
import getpass
from pyngrok import ngrok, conf
import os
import threading
from flask import Flask

# list of stocks to predict
stocks = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'META', 'TSLA', 'BRK-B', 'DIS','BTC-USD', 'ETH-USD']
today = date.today()

#Data Preprocessing

In [5]:

# dictionary to store prediction values
predictions = {}

# loop for getting predition of each stock
for stock in stocks:
    # download stock data from Yahoo Finance API
    stock_data = yf.download(stock, start='2020-01-01', end=today)
    
    # extract closing prices
    closing_Arr = stock_data['Close']
    closing_Price = closing_Arr.values

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_prices = scaler.fit_transform(closing_Price.reshape(-1,1))

    # Create sequences of length 60
    seq_length = 60
    X = []
    y = []
    for i in range(seq_length, len(scaled_prices)):
        X.append(scaled_prices[i-seq_length:i, 0])
        y.append(scaled_prices[i, 0])

    X = np.array(X)
    y = np.array(y)

    # Reshape the input data for LSTM
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=100))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=1, batch_size=32)

    # Use the model to make predictions for the next 7 days
    next_days = []
    for i in range(7):
        last_seq = closing_Price.reshape(-1,1)[-seq_length:]
        last_seq_scaled = scaler.transform(last_seq)
        next_day_scaled = model.predict(np.array([last_seq_scaled]))
        next_day = scaler.inverse_transform(next_day_scaled)[0][0]
        next_days.append(next_day)
        closing_Price = np.append(closing_Price.reshape(-1,1), [[next_day]], axis=0)
        
    dates_stock = closing_Arr.index
    last_date = dates_stock[-1]

    # get the last date in the index
    last_date = closing_Arr.index[-1]

    # create an array of the next 8 days (-1) =7
    date_range = pd.date_range(last_date, periods=8, freq='D')
    
    #removing the first day as thats today`s closing price -1
    date_range = date_range[1:]

    # convert the array to a numpy array
    date_range_array = np.array(date_range)

    Dates = np.concatenate([dates_stock, date_range_array])
    # Convert the dates to strings in the format 'YYYY-MM-DD'
    Dates = Dates.astype('datetime64[D]').astype(str)

    # Add the predicted closing prices and dates to the dictionary
    predictions[stock] = {'Dates': Dates, 'close': closing_Price.tolist()} 
    print(stock)

[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 86ms/step
AAPL
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 35ms/step
AMZN
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 55ms/step
GOOG
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 32ms/step
MSFT
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 29ms/step
META
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 29ms/step
TSLA
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 31ms/step
BRK-B
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 31ms/s

#pyngrok

In [6]:
#pyngrok to open a tunnel to that server.
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(22, "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel available, access with `ssh root@8.tcp.ngrok.io -p19578`


#Flask

In [7]:
#Flask app
os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

@app.route("/")
def index():
    # Return a list of available stock symbols
    return "Available stocks: " + ", ".join(predictions.keys())

  # return api root url / stock symbol to get specific stock data
@app.route("/<string:symbol>")
def get_stock(symbol):
    if symbol in predictions:
        data = []
        dates = predictions[symbol]['Dates'][-7:]
        prices = predictions[symbol]['close'][-7:]
        for i in range(len(dates)):
            data.append({
                "date": dates[i],
                "price": prices[i][0]
            })
        # Convert the list of dictionaries to JSON format
        json_data = json.dumps(data, default=str, indent=4)
        print(json_data)
        return json_data
    else:
        return "Invalid stock symbol"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


 * ngrok tunnel "http://9924-35-233-187-125.ngrok.io" -> "http://127.0.0.1:5000"


In [ ]:
# disconnect and kill the ngrok and flask
ngrok.disconnect(public_url)
ngrok.kill()

#END